# Aplicando modelagem de assuntos ao DHBB
Neste capítulo vamos explorar ferramentas de modelagem de assuntos e explorar aplicações ao DHBB. Como sempre, começamos com alguns imports familiares.

In [1]:
import os, glob, pickle
import spacy
from spacy import displacy
from sqlalchemy import create_engine
from dhbbmining import *
import ipywidgets as widgets
from tqdm import tqdm

Vamos também carregar o modelo de NLP para a língua portuguesa do Spacy:

In [2]:
nlp = spacy.load("pt_core_news_sm")

Agora faremos alguns imports novos, particularmente da biblioteca [Gensim](https://radimrehurek.com/gensim), que nos oferece as ferramentas que necessitamos para modelagem de assuntos.

In [3]:
from string import punctuation
from gensim.test.utils import datapath
from gensim import utils
from gensim.models import Word2Vec, word2vec

Para minimizar o uso de memória, vamos construir uma classe para representar o nosso corpus como um iterador, operando diretamente do banco de dados. Desta forma, ao fazer nossas análises, podemos carregar um documento por vez para alimentar os modelos, sem a necessidade de manter todo o corpus na memória, economizando memória RAM.

In [10]:
eng = create_engine("sqlite:///minha_tabela.sqlite")

class DHBBCorpus:
    def __init__(self, ndocs=10000):
        self.ndocs = min(7687,ndocs)
        self.counter = 1
    def __iter__(self):
        with eng.connect() as con:
            res = con.execute(f'select corpo from resultados limit {self.ndocs};')
            for doc in res:
                d = self.pre_process(doc[0])
                if self.counter%10 == 0:
                    print (f"Verbete {self.counter} de {6*self.ndocs}\r", end='')
                for s in d:
                    yield s
                self.counter += 1
    def pre_process(self, doc):
        n = nlp(doc, disable=['tagger', 'ner','entity-linker', 'textcat','entity-ruler','merge-noun-chunks','merge-entities','merge-subtokens'])
        results = []
        for sentence in n.sents:
            s = sentence.text.split()
            if not s:
                continue
            results.append([token.strip().strip(punctuation) for token in s if token.strip().strip(punctuation)])
        return results
        

Abaixo um pequeno exemplo de como a classe `DHBBCorpus` funciona:

In [111]:
DC = DHBBCorpus(5)
for d in DC:
    pass
#     print(n,d)
    

## Word2vec 
Vamos começar pelo treinamento de um modelo word2vec. Este modelo itera 6 vezez sobre o corpus logo, devemos ver o contador atingir 46122. Estas repetições são necessárias para permitir a 

In [11]:
if os.path.exists('dhbb.w2v'):
    model = Word2Vec.load('dhbb.w2v')
else:
    DC = DHBBCorpus()
    model = Word2Vec(sentences=DC, workers=32)
    model.save('dhbb.w2v')

In [12]:
model.save('dhbb.w2v')

### Visualizando os vetores de palavras

In [ ]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling


def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for word in model.wv.vocab:
        vectors.append(model.wv[word])
        labels.append(word)

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)

    # reduce using t-SNE
    vectors = np.asarray(vectors)
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(model)

def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')

In [ ]:
plot_with_plotly(x_vals, y_vals, labels)